<a href="https://colab.research.google.com/github/thofes/SJ_distance/blob/main/NN_Airtime_Landing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

Mounted at /content.gdrive


In [8]:
import json 
import numpy as np



with open('/content.gdrive/MyDrive/Try_3/dataset.json', "r") as fp:
    data = json.load(fp)

X = np.array(data["keypoints"])
y = np.array(data["labels"])

print(X.shape)
print(y.shape)

(84, 75)
(84,)


In [13]:
#Creating binary classification model 1575
!pip install -U tensorflow-addons

import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation, Dense, Conv1D, Dropout, MaxPooling1D, LSTM, Conv2D, Flatten,GlobalMaxPooling1D, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy, binary_crossentropy
from sklearn.utils import shuffle
import tensorflow_addons as tfa





def Ranger(sync_period=6,
           slow_step_size=0.5,
           learning_rate=0.001,
           beta_1=0.9,
           beta_2=0.999,
           epsilon=1e-7,
           weight_decay=0.,
           amsgrad=False,
           sma_threshold=5.0,
           total_steps=0,
           warmup_proportion=0.1,
           min_lr=0.,
           name="Ranger"):
    inner = tfa.optimizers.RectifiedAdam(learning_rate, beta_1, beta_2, epsilon, weight_decay, amsgrad, sma_threshold, total_steps, warmup_proportion, min_lr, name)
    optim = tfa.optimizers.Lookahead(inner, sync_period, slow_step_size, name)
    return optim



model = Sequential([
    
    Dense(units=1200, input_shape=(75,1), activation='relu'),                  
    Conv1D(filters=32, kernel_size=3, strides=3, padding='same',
           activation='relu'),
    Dropout(0.15),
    Conv1D(filters=64, kernel_size=3, strides=1, padding='same', 
           activation='relu'),
    Dropout(0.15),
    Flatten(),
    Dense(units=1, activation='sigmoid') 
])

save_callback = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=50,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)
def scheduler(epoche, lr):
      if epoche < 2:
          return lr
      else:
          return lr*0.97
lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)
#model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=Ranger(), loss='binary_crossentropy', metrics=['accuracy'])


model.summary()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 75, 1200)          2400      
                                                                 
 conv1d_10 (Conv1D)          (None, 25, 32)            115232    
                                                                 
 dropout_12 (Dropout)        (None, 25, 32)            0         
                                                                 
 conv1d_11 (Conv1D)          (None, 25, 64)            6208      
                                                                 
 dropout_13 (Dropout)        (None, 25, 64)            0         
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                     

In [14]:
history = model.fit(x=X, y=y, batch_size=40, validation_split = 0.1, epochs=1000, shuffle=True, verbose = 2, callbacks=[save_callback])

Epoch 1/1000
2/2 - 4s - loss: 0.6927 - accuracy: 0.5600 - val_loss: 0.6914 - val_accuracy: 0.7778 - 4s/epoch - 2s/step
Epoch 2/1000
2/2 - 0s - loss: 0.6927 - accuracy: 0.5200 - val_loss: 0.6915 - val_accuracy: 0.7778 - 152ms/epoch - 76ms/step
Epoch 3/1000
2/2 - 0s - loss: 0.6917 - accuracy: 0.6800 - val_loss: 0.6917 - val_accuracy: 0.7778 - 145ms/epoch - 73ms/step
Epoch 4/1000
2/2 - 0s - loss: 0.6927 - accuracy: 0.5733 - val_loss: 0.6930 - val_accuracy: 0.5556 - 135ms/epoch - 67ms/step
Epoch 5/1000
2/2 - 0s - loss: 0.6928 - accuracy: 0.5333 - val_loss: 0.6950 - val_accuracy: 0.0000e+00 - 144ms/epoch - 72ms/step
Epoch 6/1000
2/2 - 0s - loss: 0.6915 - accuracy: 0.6000 - val_loss: 0.6944 - val_accuracy: 0.0000e+00 - 137ms/epoch - 69ms/step
Epoch 7/1000
2/2 - 0s - loss: 0.6921 - accuracy: 0.5333 - val_loss: 0.6975 - val_accuracy: 0.0000e+00 - 160ms/epoch - 80ms/step
Epoch 8/1000
2/2 - 0s - loss: 0.6916 - accuracy: 0.5600 - val_loss: 0.6998 - val_accuracy: 0.0000e+00 - 150ms/epoch - 75ms/st